In [6]:
import json
import numpy as np
import pandas as pd
from inline_sql import sql
import matplotlib.pyplot as plt

### Importacion de tablas

In [7]:
pozos = pd.read_csv("capitulo-iv-pozos.csv")
produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")

C:\Users\carri\AppData\Local\Temp\ipykernel_13568\1836493753.py:2: DtypeWarning: Columns (16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")


In [15]:
pozos.info(), produccion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    27727 non-null  object 
 1   idpozo                   27727 non-null  int64  
 2   area                     27727 non-null  object 
 3   cod_area                 27727 non-null  object 
 4   empresa                  27727 non-null  object 
 5   yacimiento               27727 non-null  object 
 6   cod_yacimiento           27727 non-null  object 
 7   formacion                27699 non-null  object 
 8   cuenca                   27727 non-null  object 
 9   provincia                27727 non-null  object 
 10  cota                     27727 non-null  float64
 11  profundidad              27727 non-null  float64
 12  clasificacion            27727 non-null  object 
 13  subclasificacion         27727 non-null  object 
 14  tipo_recurso             27

(None, None)

### Limpieza pozos no productivos

Tomo los idpozos que tuvieron algo de producción en algún mes del año

In [9]:
produccion['index'] = produccion.index
produccion_nonula = sql^ """
                         SELECT *
                         FROM produccion AS p1
                         WHERE EXISTS ( SELECT p2.idpozo, SUM(p2.prod_pet) AS prod_pet_tot, SUM(p2.prod_gas) AS prod_gas_tot
                                        FROM produccion AS p2
                                        WHERE p1.idpozo = p2.idpozo
                                        GROUP BY p2.idpozo
                                        HAVING prod_pet_tot != 0 OR prod_gas_tot != 0
                                      )
                         """
produccion_nonula.info()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274497 entries, 0 to 274496
Data columns (total 39 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idempresa               274497 non-null  object 
 1   anio                    274497 non-null  int64  
 2   mes                     274497 non-null  int64  
 3   idpozo                  274497 non-null  int64  
 4   prod_pet                274497 non-null  float64
 5   prod_gas                274497 non-null  float64
 6   prod_agua               274497 non-null  float64
 7   iny_agua                274497 non-null  float64
 8   iny_gas                 274497 non-null  float64
 9   iny_co2                 274497 non-null  float64
 10  iny_otro                274497 non-null  float64
 11  tef                     274497 non-null  float64
 12  vida_util               90 non-null      float64
 13  tipoextraccion          274475 non-null  object 
 14  tipoestado          

Filtro en la tabla de pozos los que no tuvieron nada de producción

In [10]:
pozos = pozos[pozos['idpozo'].isin(produccion_nonula['idpozo'].unique())]
pozos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    27727 non-null  object 
 1   idpozo                   27727 non-null  int64  
 2   area                     27727 non-null  object 
 3   cod_area                 27727 non-null  object 
 4   empresa                  27727 non-null  object 
 5   yacimiento               27727 non-null  object 
 6   cod_yacimiento           27727 non-null  object 
 7   formacion                27699 non-null  object 
 8   cuenca                   27727 non-null  object 
 9   provincia                27727 non-null  object 
 10  cota                     27727 non-null  float64
 11  profundidad              27727 non-null  float64
 12  clasificacion            27727 non-null  object 
 13  subclasificacion         27727 non-null  object 
 14  tipo_recurso             27

### Separado de coordenadas en columnas nuevas

Me creo dos columnas nuevas de longitud y latitud

In [12]:
pozos['coords'] = pozos['geojson'].apply(json.loads)
pozos['coords'] = pozos['coords'].apply(lambda x: x['coordinates'])
pozos[['lon', 'lat']] = pd.DataFrame(pozos['coords'].tolist(), index=pozos.index)
pozos = pozos.drop(columns=['coords'])
pozos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    27727 non-null  object 
 1   idpozo                   27727 non-null  int64  
 2   area                     27727 non-null  object 
 3   cod_area                 27727 non-null  object 
 4   empresa                  27727 non-null  object 
 5   yacimiento               27727 non-null  object 
 6   cod_yacimiento           27727 non-null  object 
 7   formacion                27699 non-null  object 
 8   cuenca                   27727 non-null  object 
 9   provincia                27727 non-null  object 
 10  cota                     27727 non-null  float64
 11  profundidad              27727 non-null  float64
 12  clasificacion            27727 non-null  object 
 13  subclasificacion         27727 non-null  object 
 14  tipo_recurso             27

# Unicidad sigla-formacion

Veo duplicados en sigla y formacion. Un duplicado difiere en la provincia (Santa Cruz / Tierra del fuego), o difiere en la empresa. Las producciones son practicamente las mismas, muy parecidas, podemos decir que los duplicados son el mismo pozo.

In [34]:
duplicados = pozos[pozos[['sigla','formacion']].duplicated(keep=False)]
duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 3381 to 60478
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    80 non-null     object 
 1   idpozo                   80 non-null     int64  
 2   area                     80 non-null     object 
 3   cod_area                 80 non-null     object 
 4   empresa                  80 non-null     object 
 5   yacimiento               80 non-null     object 
 6   cod_yacimiento           80 non-null     object 
 7   formacion                80 non-null     object 
 8   cuenca                   80 non-null     object 
 9   provincia                80 non-null     object 
 10  cota                     80 non-null     float64
 11  profundidad              80 non-null     float64
 12  clasificacion            80 non-null     object 
 13  subclasificacion         80 non-null     object 
 14  tipo_recurso             80

# Unicidad de coordenadas

Me fijo geoms repetidos

In [17]:
geom_dup = pozos[pozos['geom'].duplicated(keep=False)]
geom_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2387 entries, 239 to 84298
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    2387 non-null   object 
 1   idpozo                   2387 non-null   int64  
 2   area                     2387 non-null   object 
 3   cod_area                 2387 non-null   object 
 4   empresa                  2387 non-null   object 
 5   yacimiento               2387 non-null   object 
 6   cod_yacimiento           2387 non-null   object 
 7   formacion                2387 non-null   object 
 8   cuenca                   2387 non-null   object 
 9   provincia                2387 non-null   object 
 10  cota                     2387 non-null   float64
 11  profundidad              2387 non-null   float64
 12  clasificacion            2387 non-null   object 
 13  subclasificacion         2387 non-null   object 
 14  tipo_recurso             2

Me fijo geojsons repetidos

In [18]:
geojson_dup = pozos[pozos['geojson'].duplicated(keep=False)]
geojson_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2387 entries, 239 to 84298
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    2387 non-null   object 
 1   idpozo                   2387 non-null   int64  
 2   area                     2387 non-null   object 
 3   cod_area                 2387 non-null   object 
 4   empresa                  2387 non-null   object 
 5   yacimiento               2387 non-null   object 
 6   cod_yacimiento           2387 non-null   object 
 7   formacion                2387 non-null   object 
 8   cuenca                   2387 non-null   object 
 9   provincia                2387 non-null   object 
 10  cota                     2387 non-null   float64
 11  profundidad              2387 non-null   float64
 12  clasificacion            2387 non-null   object 
 13  subclasificacion         2387 non-null   object 
 14  tipo_recurso             2

In [21]:
# Checkeo geom = geojson
False in (geom_dup.index == geojson_dup.index)

False

In [20]:
# Cantidad de coordenadas unicas
len(geom_dup['geom'].unique())

1092

### Ejemplo duplicado

In [22]:
geom_dup[(geom_dup['geom'] == geom_dup['geom'].iloc[500])]

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
42188,YPF.Nq.EL-22,109903,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,rayoso,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,-69.275719,-37.361085
42189,YPF.Nq.EL-22,109904,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,-69.275719,-37.361085


In [23]:
sample_dup = produccion[(produccion['idpozo'] == 109903) | (produccion['idpozo'] == 109904)]
sample_dup[(sample_dup['prod_pet'] != 0) & (sample_dup['prod_gas'] != 0)]

,idempresa,anio,mes,idpozo,prod_pet,prod_gas,prod_agua,iny_agua,iny_gas,iny_co2,...,areayacimiento,cuenca,provincia,tipo_de_recurso,proyecto,clasificacion,subclasificacion,sub_tipo_recurso,fecha_data,index
36661,YPF,2024,1,109904,15.78,0.07,161.95,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36661
36662,YPF,2024,1,109903,36.83,0.15,377.89,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36662
75000,YPF,2024,2,109904,13.38,0.06,157.97,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75000
75001,YPF,2024,2,109903,31.22,0.15,368.58,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75001
112625,YPF,2024,3,109904,15.13,0.07,172.38,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112625
112626,YPF,2024,3,109903,35.30,0.16,402.23,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112626
154639,YPF,2024,4,109904,15.16,0.07,174.34,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154639
154640,YPF,2024,4,109903,35.38,0.16,406.80,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154640
190633,YPF,2024,5,109904,12.93,0.06,153.19,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190633
190634,YPF,2024,5,109903,30.18,0.14,357.43,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190634


In [14]:
geom_dup.loc[42188] == geom_dup.loc[42189]

sigla                       True
idpozo                     False
area                        True
cod_area                    True
empresa                     True
yacimiento                  True
cod_yacimiento              True
formacion                  False
cuenca                      True
provincia                   True
cota                        True
profundidad                 True
clasificacion               True
subclasificacion            True
tipo_recurso                True
sub_tipo_recurso            True
gasplus                     True
tipopozo                    True
tipoextraccion              True
tipoestado                  True
adjiv_fecha_inicio_perf     True
adjiv_fecha_fin_perf        True
adjiv_fecha_inicio_term     True
adjiv_fecha_fin_term        True
geojson                     True
geom                        True
lon                        False
lat                        False
dtype: bool

### Checkeo cuáles tienen bocas unicas

Me fijo cuáles tienen bocas unicas dentro de mismas coordenadas. Esperaríamos que no haya ninguna ya que una boca es un punto fisico en un terreno, por lo que no debería existir una boca unica en una lista de coordenadas repetidas. Sin embargo hay 155

In [85]:
geom_dup_uneqbocas = geom_dup[~geom_dup['sigla'].duplicated(keep=False)]
geom_dup_uneqbocas.to_csv("duplicados_eqcoord_uneqsigla.csv")
geom_dup_uneqbocas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 655 to 83928
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    155 non-null    object 
 1   idpozo                   155 non-null    int64  
 2   area                     155 non-null    object 
 3   cod_area                 155 non-null    object 
 4   empresa                  155 non-null    object 
 5   yacimiento               155 non-null    object 
 6   cod_yacimiento           155 non-null    object 
 7   formacion                155 non-null    object 
 8   cuenca                   155 non-null    object 
 9   provincia                155 non-null    object 
 10  cota                     155 non-null    float64
 11  profundidad              155 non-null    float64
 12  clasificacion            155 non-null    object 
 13  subclasificacion         155 non-null    object 
 14  tipo_recurso             15

Un ejemplo, difiere solo en formacion y sigla, es posible que hayan anotado diferente al anotar distintas formaciones

In [84]:
geom_dup[geom_dup['geom'] == geom_dup_uneqbocas['geom'].iloc[55]]

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
60315,AEA.NQ.Gu-1169,129645,AL NORTE DE LA DORSAL,NDD,YPF S.A.,GUANACO,GUA,lotena,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2008-02-23,2008-03-05,2008-03-05,2008-03-18,"{""type"":""Point"",""coordinates"":[-69.23576737000...",0101000020E61000004BE705D0164F51C0885DC23E0D6F...,-69.235767,-38.867592
76493,Apa.Nq.Gu-1169,157601,AL NORTE DE LA DORSAL,NDD,YPF S.A.,GUANACO,GUA,quintuco,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2008-02-23,2008-03-05,2008-03-06,2008-03-07,"{""type"":""Point"",""coordinates"":[-69.23576737000...",0101000020E61000004BE705D0164F51C0885DC23E0D6F...,-69.235767,-38.867592


# Unicidad de sigla

Me fijo de los que tienen distintas coordenadas, cuales tienen misma boca. Esperaríamos que no haya ninguna por lo mismo de antes, si no hay coordenadas repetidas, en principio no debería haber dos siglas iguales. Sin embargo hay 294

In [46]:
sigla_dup = pozos[(~pozos['geom'].duplicated(keep=False))]
sigla_dup = sigla_dup[(sigla_dup['sigla'].duplicated(keep=False))]
sigla_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 224 to 84301
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sigla                    294 non-null    object 
 1   idpozo                   294 non-null    int64  
 2   area                     294 non-null    object 
 3   cod_area                 294 non-null    object 
 4   empresa                  294 non-null    object 
 5   yacimiento               294 non-null    object 
 6   cod_yacimiento           294 non-null    object 
 7   formacion                294 non-null    object 
 8   cuenca                   294 non-null    object 
 9   provincia                294 non-null    object 
 10  cota                     294 non-null    float64
 11  profundidad              294 non-null    float64
 12  clasificacion            294 non-null    object 
 13  subclasificacion         294 non-null    object 
 14  tipo_recurso             29

Un ejemplo, tienen distinta empresa y distinta formacion y area, el cod_yac de uno parece haber sido eliminado sin embargo es el mismo

In [73]:
dup = sigla_dup[sigla_dup['sigla'] == sigla_dup['sigla'].iloc[50]]
dup

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
3380,YPF.Nq.EL-1,8911,EL TRAPIAL ESTE,ETE,CHEVRON ARGENTINA S.R.L.,EL LIMITE,ELIM,huitrín,NEUQUINA,Neuquén,...,Electrosumergible,Extracción Efectiva,1995-07-07,1995-07-18,1995-07-22,1995-07-30,"{""type"":""Point"",""coordinates"":[-69.21550000000...",0101000020E61000006F1283C0CA4D51C035EF384547AE...,-69.215500,-37.361550
44174,YPF.Nq.EL-1,112143,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Electrosumergible,Extracción Efectiva,1995-07-07,1995-07-18,1995-07-22,1995-07-30,"{""type"":""Point"",""coordinates"":[-69.21645929000...",0101000020E6100000C9DD1078DA4D51C01835A9B239AE...,-69.216459,-37.361136


Nos fijamos isclose con lon y lat, viendolos a mano son parecidos pero isclose da Falso, por ahí habría que ajustarlo. Ya lo ajuste a atol=1e-4 ahora isclose la latitud pero no isclose la longitud (antes ninguna de las dos eraclose xd)

In [79]:
np.isclose(dup['lon'], dup['lon'].iloc[0], atol=1e-4), np.isclose(dup['lat'], dup['lat'].iloc[0], atol=1e-4)

(array([ True, False]), array([ True,  True]))

Funcion que me separa los ids de los que no isclose me fijo y son 22, habría que revisar a mano con los de su misma sigla en "sigla_dup".

In [80]:
def coords_notclose (df):
    siglas = df['sigla'].unique()
    ids = []
    
    for sigla in siglas:
        pozos_eqsigla = df[df['sigla'] == sigla]
        lon_close = np.isclose(pozos_eqsigla['lon'], pozos_eqsigla['lon'].iloc[0], atol=1e-4)
        lat_close = np.isclose(pozos_eqsigla['lat'], pozos_eqsigla['lat'].iloc[0], atol=1e-4)
        
        for p in range (len(lon_close)):
            if not (lon_close[p] & lat_close[p]):
                ids.append(pozos_eqsigla['idpozo'].iloc[p])
                
    return ids

sigla_dup[sigla_dup['idpozo'].isin(coords_notclose(sigla_dup))]

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
4614,SJ.RN.LN-7,10144,LOMA NEGRA,LNE,CAPEX S.A.,LOMA NEGRA,LNE,precuyo,NEUQUINA,Rio Negro,...,Electrosumergible,Extracción Efectiva,1998-10-19,1998-11-09,1999-02-09,1999-02-20,"{""type"":""Point"",""coordinates"":[-67.64238299999...",0101000020E6100000682096CD1CE950C06C5CFFAECF60...,-67.642383,-38.756338
4687,AEC.RN.PP-3,10218,PUESTO PRADO,PUPR,PRESIDENT PETROLEUM S.A.,PUESTO PRADO,PPR,precuyo,NEUQUINA,Rio Negro,...,Bombeo Mecánico,Extracción Efectiva,NaN,NaN,NaN,NaN,"{""type"":""Point"",""coordinates"":[-67.42941000000...",0101000020E6100000D3A414747BDB50C0327216F6B44B...,-67.429410,-38.591460
4688,AEC.RN.PP-3,10219,PUESTO PRADO,PUPR,PRESIDENT PETROLEUM S.A.,PUESTO PRADO,PPR,punta rosada,NEUQUINA,Rio Negro,...,Bombeo Mecánico,Extracción Efectiva,NaN,NaN,NaN,NaN,"{""type"":""Point"",""coordinates"":[-67.42995999999...",0101000020E6100000A4A5F27684DB50C0E449D235934B...,-67.429960,-38.590430
4697,AEC.RN.PP-8,10228,PUESTO PRADO,PUPR,PRESIDENT PETROLEUM S.A.,PUESTO PRADO,PPR,punta rosada,NEUQUINA,Rio Negro,...,Bombeo Mecánico,Extracción Efectiva,NaN,NaN,NaN,NaN,"{""type"":""Point"",""coordinates"":[-67.42448000000...",0101000020E6100000977329AE2ADB50C03BFC3559A34A...,-67.424480,-38.583110
42197,YPF.Nq.EL-27,109913,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Parado Transitoriamente,2004-07-15,2004-07-31,2004-08-14,2004-08-27,"{""type"":""Point"",""coordinates"":[-69.21928241000...",0101000020E610000070E216B9084E51C03532C0EC3BAE...,-69.219282,-37.361204
44174,YPF.Nq.EL-1,112143,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Electrosumergible,Extracción Efectiva,1995-07-07,1995-07-18,1995-07-22,1995-07-30,"{""type"":""Point"",""coordinates"":[-69.21645929000...",0101000020E6100000C9DD1078DA4D51C01835A9B239AE...,-69.216459,-37.361136
44175,YPF.Nq.EL-2,112144,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,1995-07-18,1995-07-25,1995-07-31,1995-08-09,"{""type"":""Point"",""coordinates"":[-69.22208639999...",0101000020E6100000BA38E0A9364E51C000757FCC39AE...,-69.222086,-37.361139
44178,YPF.Nq.EL-4,112147,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Parado Transitoriamente,1995-07-30,1995-08-06,1995-08-14,1995-08-19,"{""type"":""Point"",""coordinates"":[-69.23345100000...",0101000020E6100000FC8D76DCF04E51C0E140ED5C3DAE...,-69.233451,-37.361248
44179,YPF.Nq.EL-5,112148,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Cavidad Progresiva,Parado Transitoriamente,1995-08-26,1995-09-01,1995-09-11,1995-09-15,"{""type"":""Point"",""coordinates"":[-69.23857533999...",0101000020E6100000A7BB80D1444F51C0603B39BF38AE...,-69.238575,-37.361107
44180,YPF.Nq.EL-6,112149,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Electrosumergible,Extracción Efectiva,1995-08-06,1995-08-15,1995-08-21,1995-08-27,"{""type"":""Point"",""coordinates"":[-69.24463187000...",0101000020E6100000664D6E0CA84F51C0DBC1EBA73DAE...,-69.244632,-37.361257
